<a href="https://colab.research.google.com/github/aidot-kr/AISecurity/blob/master/6_TopicModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Topic Modeling
- 참고DataSet: 네이버 영화리뷰, 해당 폴더에서 ratings.txt 다운로드
- 소스코드 출처: https://coredottoday.github.io/

1) 다운로드된 파일읽기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/My\ Drive/AISecurity

/content/drive/My Drive/AISecurity


In [ ]:
ls

2.TFIDF.ipynb           5.Word2Vec-FastText.ipynb  ratings.txt
3.SimpleCrawling.ipynb  6.Similarity.ipynb         Sample.ipynb
4_MeCab.ipynb           6.TopicModeling2.ipynb     spam.csv
4_Pre_Processing.ipynb  6.TopicModeling.ipynb
5.OneHotEncode.ipynb    7.ngram.ipynb


In [ ]:
# Colab 서버스펙확인, GPU 할당을 위해서는 메뉴 > 런타임 > 런타임유형변경에서 GPU 선택
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12868769792360832107, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9078734879070652370
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 256433079731021294
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15473775744
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13146161969386282878
 physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:04.0, compute capability: 7.0"]

In [ ]:
import pandas as pd

# ratings.txt는 탭구분자를 기준으로 읽기 실행
doc = pd.read_csv('ratings.txt',sep = '\t' ) 

In [ ]:
# ratings.txt를 file open Read도 가능
f = open('ratings.txt', 'r',  encoding='utf-8')
data = f.read()
f.close()

In [ ]:
print(len(data))

9248467


In [ ]:
doc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          10000 non-null  int64 
 1   document    10000 non-null  object
 2   label       10000 non-null  int64 
 3   clean_doc2  10000 non-null  object
 4   clean_doc   10000 non-null  object
dtypes: int64(2), object(3)
memory usage: 468.8+ KB


In [ ]:
doc.head(5)

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [ ]:
doc = doc[doc['document'].notnull()]  # Null값이 있는 필드는 제외
doc = doc[:50000] # 빠른 실습을 위해서 50000개의 Row만 추출

In [ ]:
doc.isnull().sum() #null 포함여부 체크

id          0
document    0
label       0
dtype: int64

2) 텍스트데이터 전처리
- 형태소를 사용해서 명사만 추출한다.
- 정확도 개선을 위해서는 불용어, 원형복원, 동철이의어 처리가 필요함

In [ ]:
# 한글 불용어 처리를 위한 함수
import re
def clean_sentense(txt):
    pattern = '(\d\d\d-\d\d\d\d-\d\d\d\d)' # 전화번호 제거 (000-0000-0000),\d: 숫자 1개
    txt = re.sub(pattern=pattern, repl='', string=txt)          
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거, a-z 사이의 문자, 
    txt = re.sub(pattern=pattern, repl='', string=txt)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    txt = re.sub(pattern=pattern, repl='', string=txt)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    txt = re.sub(pattern=pattern, repl='', string=txt)
    pattern = '<[^>]*>'         # HTML 태그 제거
    txt = re.sub(pattern=pattern, repl='', string=txt)
    pattern = '[^\w\s]'         # 특수기호제거
    txt = re.sub(pattern=pattern, repl='', string=txt)
    
    return txt   

In [ ]:
!pip3 install konlpy

     |████████████████████████████████| 19.4MB 72.8MB/s 
     |████████████████████████████████| 92kB 11.0MB/s 
     |████████████████████████████████| 3.8MB 59.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
from konlpy.tag import Okt
okt = Okt()

# 문장 추출하는 함수 (한 글자는 제외)
def extWord(x):
    texts = okt.morphs(x, norm=True, stem=True)
    #texts = okt.nouns(x)
    texts_out = []
    for sent in texts:        
        if len(sent) > 1:
            texts_out.append(sent)
    return texts_out

In [ ]:
doc['clean_doc2'] = doc['document'].apply(lambda x: clean_sentense(x))   
doc['clean_doc'] = doc['clean_doc2'].apply(lambda x: extWord(x))   

In [ ]:
doc.head(20)


,id,document,label,clean_doc2,clean_doc
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1,어릴때보고 지금다시봐도 재밌어요,"[어리다, 보고, 지금, 다시, 보다, 재밌다]"
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,"[디자인, 배우다, 학생, 으로, 외국, 디자이너, 일군, 전통, 통해, 발전, 하..."
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음 최고,"[폴리스스토리, 시리즈, 부터, 까지, 버리다, 하나, 없다, 최고]"
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,"[오다, 연기, 진짜, 쩔다, 지루하다, 생각, 하다, 몰입, 하다, 보다, 그렇다..."
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,"[안개, 자욱하다, 밤하늘, 뜨다, 있다, 초승달, 같다, 영화]"
5,2190435,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,"[사랑, 해보다, 사람, 라면, 처음, 부터, 까지, 웃다, 있다, 영화]"
6,9279041,완전 감동입니다 다시봐도 감동,1,완전 감동입니다 다시봐도 감동,"[완전, 감동, 이다, 다시, 보다, 감동]"
7,7865729,개들의 전쟁2 나오나요? 나오면 1빠로 보고 싶음,1,개들의 전쟁2 나오나요 나오면 1빠로 보고 싶음,"[전쟁, 나오다, 나오다, 보고, 싶다]"
8,7477618,굿,1,굿,[]
9,9250537,바보가 아니라 병 쉰 인듯,1,바보가 아니라 병 쉰 인듯,"[바보, 아니다, 이다]"


3) gensim을 사용한 인코딩 처리
- 토큰화된 문서를 기반으로 각 단어를 정수로 인코딩
- 각 문서내의 단어의 빈도수를 구함
- 예시) dictionary[1]은 정수인코딩 1번으로 처리된 단어를 의미, corpus[1] 1번 문서의 단어 빈도임

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(doc['clean_doc'])
corpus = [dictionary.doc2bow(text) for text in doc['clean_doc']]
print(dictionary[1], corpus[1])

보고 [(6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 2), (32, 1), (33, 1), (34, 1), (35, 1)]


In [ ]:
#  문서와 코퍼스의 빈도를 기준 출력
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus[:100]]

[[('다시', 1), ('보고', 1), ('보다', 1), ('어리다', 1), ('재밌다', 1), ('지금', 1)],
 [('감사하다', 1),
  ('같다', 2),
  ('까지', 1),
  ('꾸다', 1),
  ('나가다', 1),
  ('노라노', 1),
  ('디자이너', 1),
  ('디자인', 1),
  ('문화', 1),
  ('발전', 1),
  ('배우다', 1),
  ('부럽다', 1),
  ('사람', 1),
  ('사실', 1),
  ('산업', 1),
  ('시절', 1),
  ('어렵다', 1),
  ('에서도', 1),
  ('열정', 1),
  ('외국', 1),
  ('우리나라', 1),
  ('으로', 1),
  ('이루다', 1),
  ('일군', 1),
  ('있다', 2),
  ('전통', 2),
  ('지키다', 1),
  ('통해', 1),
  ('하다', 1),
  ('학생', 1)],
 [('까지', 1),
  ('버리다', 1),
  ('부터', 1),
  ('시리즈', 1),
  ('없다', 1),
  ('최고', 1),
  ('폴리스스토리', 1),
  ('하나', 1)],
 [('보다', 1),
  ('하다', 2),
  ('그렇다', 1),
  ('몰입', 1),
  ('생각', 1),
  ('연기', 1),
  ('영화', 1),
  ('오다', 1),
  ('이렇다', 1),
  ('지루하다', 1),
  ('진짜', 2),
  ('쩔다', 1)],
 [('같다', 1),
  ('있다', 1),
  ('영화', 1),
  ('뜨다', 1),
  ('밤하늘', 1),
  ('안개', 1),
  ('자욱하다', 1),
  ('초승달', 1)],
 [('까지', 1),
  ('사람', 1),
  ('있다', 1),
  ('부터', 1),
  ('영화', 1),
  ('라면', 1),
  ('사랑', 1),
  ('웃다', 1),
  ('처음', 1),
  ('해보다', 1)],
 [('다시', 1

In [ ]:
len(dictionary)

25904

4) 토픽모델 구축(LDA Model 학습)
- corpus : 문서 벡터
- id2word : 단어ID와 매핑된 단어의 빈도
- num_topics : 가설 토픽 개수
- chunksize : 훈련 알고리즘에 사용되는 문서사이즈, 빠른 학습을 위해서 사이즈 상향 
- Hoffman의 논문에 의하면 Chunksize는 모델 품질에 영향은 일부 있음
- pass : epochs와 같은 용어로 전체 코퍼스에서 모델을 학습시키는 빈도
- iteration : 각각 문서에 대한 반복횟수, passes & iteration 은 많을수록 좋음
- per_word_topics : 각 단어에 대해 가장 가능성이 높은 주제의 내림차순으로 토픽목록 x 피처길이 = 토픽값 계산 

In [ ]:
import gensim
from gensim.models.ldamodel import LdaModel

num_topics = 10 # 가설 토픽개수
chunksize = 2000 # 훈련알고리즘에 사용되는 문서의 개수
passes = 20 # 모델 학습빈도
iterations = 400 # 문서 반복횟수
per_word_topics = True # 가능성 높은 주제로 정렬하여 계산

%time lda_model = LdaModel(corpus = corpus, id2word = dictionary,\
                           iterations = iterations, num_topics = num_topics, chunksize = chunksize, passes = passes, per_word_topics = per_word_topics )

CPU times: user 3min 38s, sys: 271 ms, total: 3min 38s
Wall time: 3min 37s


5) gensim의 CoherenceModel을 사용한 성능평가

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

cm = CoherenceModel(model=lda_model, corpus=corpus, coherence='u_mass')
coherence = cm.get_coherence()

# 계산모델의 난이도로 해당 확률 모델의 예측정확도 판정, 수치가 낮을수록 정확성이 높음
print("Perplexity: ", lda_model.log_perplexity(corpus)) 
# 추론된 토픽간의 일관성, 높을수록 의미론적 일관성 높음
print("\nCoherence: ",coherence)

Perplexity:  -7.7981201030374

Coherence:  -4.700033749361276


6) epoch 반복에 따른 coherence 의 변화식별

In [ ]:
# import warnings
# import time

# cohrences=[]
# perplexities=[]
# passes=[]
# warnings.filterwarnings('ignore')

# for i in range(10):
    
#     ntopics, nwords = 200, 100
#     if i==0:
#         p=1
#     else:
#         p=i*20
#     tic = time.time()
#     lda4 = LdaMulticore(corpus, id2word=dictionary, num_topics=ntopics, iterations=400, passes=p)
#     print('epoch',p,time.time() - tic)
    
#     cm = CoherenceModel(model=lda4, corpus=corpus, coherence='u_mass')
#     coherence = cm.get_coherence()
#     print("Coherence",coherence)
#     coherences.append(coherence)
#     print('Perplexity: ', lda4.log_perplexity(corpus),'\n\n')
#     perplexities.append(lda4.log_perplexity(corpus))

7) 토픽결과 분석 및 추론

In [ ]:
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.132*"최고" + 0.042*"영화" + 0.037*"드라마" + 0.030*"이다" + 0.028*"인생" + 0.017*"하나" + 0.015*"정말" + 0.015*"최고다" + 0.015*"멋지다" + 0.008*"진짜"')
(1, '0.050*"하다" + 0.045*"사랑" + 0.015*"되다" + 0.012*"우리" + 0.011*"남자" + 0.011*"예쁘다" + 0.011*"에게" + 0.011*"가족" + 0.009*"되어다" + 0.009*"있다"')
(2, '0.032*"마지막" + 0.024*"장면" + 0.023*"감동" + 0.019*"슬프다" + 0.016*"울다" + 0.015*"반전" + 0.015*"이다" + 0.013*"눈물" + 0.012*"필요없다" + 0.009*"충격"')
(3, '0.129*"보다" + 0.086*"재밌다" + 0.041*"너무" + 0.035*"재미있다" + 0.026*"정말" + 0.026*"영화" + 0.024*"진짜" + 0.015*"평점" + 0.014*"좋다" + 0.013*"다시"')
(4, '0.040*"괜찮다" + 0.034*"명작" + 0.016*"이다" + 0.011*"스릴러" + 0.010*"굳다" + 0.010*"공포" + 0.010*"걸작" + 0.009*"이상" + 0.009*"꿀잼" + 0.008*"나름"')
(5, '0.093*"하다" + 0.054*"영화" + 0.041*"있다" + 0.038*"생각" + 0.021*"사람" + 0.021*"보다" + 0.019*"없다" + 0.016*"되다" + 0.013*"10" + 0.013*"싶다"')
(6, '0.093*"영화" + 0.059*"보다" + 0.024*"하다" + 0.021*"않다" + 0.016*"이다" + 0.015*"이렇다" + 0.011*"좋다" + 0.011*"정말" + 0.010*"아니다" + 0.010*"있다"')
(7, '0.023*"대박" + 0.021*"역시" + 0.016*"무

In [ ]:
for i, topic_list in enumerate(lda_model[corpus]):
    if i==10:
        break
    print('\n',doc['document'][i])
    print(doc['clean_doc'][i])
    print(i,'번째 문서의 topic 비율은','\n',topic_list)


 어릴때보고 지금다시봐도 재밌어요ㅋㅋ
['어리다', '보고', '지금', '다시', '보다', '재밌다']
0 번째 문서의 topic 비율은 
 ([(0, 0.014285714), (1, 0.014286031), (2, 0.014285925), (3, 0.8714243), (4, 0.014285714), (5, 0.014287285), (6, 0.014287584), (7, 0.014285714), (8, 0.014285714), (9, 0.014285953)], [(0, [3]), (1, [3]), (2, [3]), (3, [3]), (4, [3]), (5, [3])], [(0, [(3, 0.99999577)]), (1, [(3, 0.9999909)]), (2, [(3, 0.99999595)]), (3, [(3, 0.99999934)]), (4, [(3, 1.0)]), (5, [(3, 0.99998885)])])

 디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.
['디자인', '배우다', '학생', '으로', '외국', '디자이너', '일군', '전통', '통해', '발전', '하다', '문화', '산업', '부럽다', '사실', '우리나라', '에서도', '어렵다', '시절', '까지', '열정', '지키다', '노라노', '같다', '전통', '있다', '같다', '사람', '꾸다', '이루다', '나가다', '있다', '감사하다']
1 번째 문서의 topic 비율은 
 ([(1, 0.118270226), (2, 0.032393932), (5, 0.7401984), (9, 0.09148866)], [(6, [5]), (7, [5, 9, 1]), (8, [5, 9]), (9, [2]), (10, [1]), (11, [5]), (12, [5]), (13, [5

In [ ]:
import pprint
pprint.pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.132*"최고" + 0.042*"영화" + 0.037*"드라마" + 0.030*"이다" + 0.028*"인생" + '
  '0.017*"하나" + 0.015*"정말" + 0.015*"최고다" + 0.015*"멋지다" + 0.008*"진짜"'),
 (1,
  '0.050*"하다" + 0.045*"사랑" + 0.015*"되다" + 0.012*"우리" + 0.011*"남자" + '
  '0.011*"예쁘다" + 0.011*"에게" + 0.011*"가족" + 0.009*"되어다" + 0.009*"있다"'),
 (2,
  '0.032*"마지막" + 0.024*"장면" + 0.023*"감동" + 0.019*"슬프다" + 0.016*"울다" + '
  '0.015*"반전" + 0.015*"이다" + 0.013*"눈물" + 0.012*"필요없다" + 0.009*"충격"'),
 (3,
  '0.129*"보다" + 0.086*"재밌다" + 0.041*"너무" + 0.035*"재미있다" + 0.026*"정말" + '
  '0.026*"영화" + 0.024*"진짜" + 0.015*"평점" + 0.014*"좋다" + 0.013*"다시"'),
 (4,
  '0.040*"괜찮다" + 0.034*"명작" + 0.016*"이다" + 0.011*"스릴러" + 0.010*"굳다" + '
  '0.010*"공포" + 0.010*"걸작" + 0.009*"이상" + 0.009*"꿀잼" + 0.008*"나름"'),
 (5,
  '0.093*"하다" + 0.054*"영화" + 0.041*"있다" + 0.038*"생각" + 0.021*"사람" + 0.021*"보다" '
  '+ 0.019*"없다" + 0.016*"되다" + 0.013*"10" + 0.013*"싶다"'),
 (6,
  '0.093*"영화" + 0.059*"보다" + 0.024*"하다" + 0.021*"않다" + 0.016*"이다" + '
  '0.015*"이렇다" + 0.011*"좋다" + 0.011*"정말" + 0.01

8) 토픽키워드 시각화
- pyLDAvis 패키지를 사용하며, jupyter notebook에서 잘 동작함

In [ ]:
!pip3 install pyLDAvis

     |████████████████████████████████| 1.6MB 7.8MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=ae691c6619ee8dadc5ddd5d848e9d32f5bd8f1bab47f8e88eab7c484914ee6f5
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)